In [1]:
import os
import json
import pandas as pd
import traceback
from dotenv import load_dotenv
import ollama  # Import Ollama

In [3]:
from langchain_ollama.chat_models import ChatOllama

In [ ]:
llm = ChatOllama(
    model="llama3.1",
    temperature=0.5
)